# Graph Convolutional Neural Network model

This model is based on Convolutional Neural Networks where input molecules are modelled as undirected 2D graphs, using the pytorch library

The model takes SMILES of any query molecules as input and returns the predicted probability of binding to each non-orphan TAS2R receptor and explanations generated by UGrad-CAM

***

<div class="alert alert-block alert-warning">
Before running the following code, please make sure to have <b>all the required libraries</b>. Instruction how to obtain the full environment are present in the <b>README file</b> of this repository
</div>

***

Import the libraries and the functions from the main script

In [1]:
import os, shutil
import pandas as pd
import numpy as np
from rdkit import Chem
from chembl_structure_pipeline import standardizer
import torch
from torch_geometric.loader import DataLoader
from GCN_Eval import MolDataset_exp, BitterGCN, ugrad_cam
from Virtuous import TestAD
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.rdmolops import RemoveAllHs
from rdkit_heatmaps import mapvalues2mol
from rdkit_heatmaps.utils import transform2png

[12:26:18] Initializing Normalizer


ModuleNotFoundError: No module named 'torch'

Insert the input molecule

In [ ]:
smiles = 'CC(CC1=CC2=C(C=C1)OCO2)NC'

Set TRUE to change the naming of the molecules from SMILES to "molecule_n"

In [ ]:
# Overrides naming of molecules as molecule_N, if False the standardized SMILES will be used
NAME_OVERRIDE = False

Set TRUE to plot and save a UGrad-CAM plot for every pair. If FALSE, it will only generate UGrad-CAM plots if there is only <b>ONE</b> input molecule

<div class="alert alert-block alert-danger">
<b> ATTENTION: </b> Activating UGrad-CAM for larger queries (>10 molecules) is not recommeded due to its high computational cost
<div>

In [ ]:
# Activates plotting of explanations with UGrad-CAM
PLOT_UGRADCAM = False

Should the model check if any of the input molecules is already present in the dataset?
<br>
If True, for any known pair the results will show the ground truth (0 or 1) and not model's prediction.

In [ ]:
# If TRUE checks if evaluated pairs have in-vitro verified data and overwrites prediction with Ground Truth
GT = True

Run SMILES Standardization and data preparation

In [ ]:
# Delete files from previous runs
if os.path.exists('./src/eval/processed'):
    shutil.rmtree('./src/eval/processed')
os.makedirs('./src/eval/processed')
if os.path.exists('./src/eval/raw'):
    shutil.rmtree('./src/eval/raw')
os.makedirs('./src/eval/raw')

input_molecules = smiles

# CHECK if only one smile was given instead of a list
if type(input_molecules) != list:
    input_molecules = [input_molecules]

# Standardize molecules
def Std(input_smiles):
    mol_list = [Chem.MolFromSmiles(mol) for mol in input_smiles]
    std_mol_list = [standardizer.standardize_mol(mol) for mol in mol_list]
    parent = [standardizer.get_parent_mol(std_mol)[0] for std_mol in std_mol_list]
    parent_smi = [Chem.MolToSmiles(parent) for parent in parent]

    return parent_smi

molecules = Std(input_molecules)

# write a file of name Input_data.csv in raw
# the format is ready to be predicted by the model
with open('./src/eval/raw/Input_data.csv', 'w') as f:
    f.write(',1,3,4,5,7,8,9,10,13,14,16,38,39,40,41,42,43,44,46,47,49,50\n')
    recs = np.zeros(22,dtype=int)
    rec2idx = {'1':0,'3':1,'4':2,'5':3,'7':4,'8':5,'9':6,'10':7,'13':8,'14':9,
               '16':10,'38':11,'39':12,'40':13,'41':14,'42':15,'43':16,'44':17,
               '46':18,'47':19,'49':20,'50':21}
    for m in molecules:
        for r in range(22):
            recs[r] = 1
            f.write(m+','+','.join(recs.astype(str))+'\n')
            recs[r] = 0

Run the evaluation task over the input molecule for every non-orphan receptors with the trained model

In [ ]:
data = MolDataset_exp(root='./src/eval')
test_loader =  DataLoader(data, batch_size=22, shuffle=False, num_workers=0, persistent_workers=False)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
best_model = './src/GCN_model.pt'
model = BitterGCN(data.num_node_features - 22, data.num_edge_features)
model.load_state_dict(torch.load(best_model,map_location=torch.device('cpu')),strict=False)
model = model.to(device)

model.eval()

receptors = [1, 3, 4, 5, 7, 8, 9, 10, 13, 14, 16, 38, 39, 40, 41, 42, 43, 44, 46, 47, 49, 50]

# Create df for final predictions
final_results_df = pd.DataFrame(np.zeros(shape=(len(molecules),len(receptors))) ,columns=receptors)

## UGrad-CAM

Plot the gradient activation with UGrad-CAM, where Red is associated with activation of the node toward class 1, and Blue toward class 0

Choose a receptor to investigate, for example TAS2R14

In [ ]:
rec_to_inv = 14

In [ ]:
from IPython.display import display
# Function to Plot on Molecule
def plot_on_mol(mol,name,receptor,pred,activations, gradients):
    mol = Chem.MolFromSmiles(mol)
    test_mol = Draw.PrepareMolForDrawing(mol)
    test_mol = RemoveAllHs(test_mol) #IMPORTANT, RDKit PrepareMolForDrawing adds chiral Hs (bugged), so this patches the issue

    # Plot UGrad-CAM
    ugrad_cam_weights = ugrad_cam(mol, activations, gradients)
    colorscale = 'bwr_r' if pred == 0 else 'bwr'
    atom_weights = ugrad_cam_weights
    bond_weights = np.zeros(len(test_mol.GetBonds()))
    limit=[max(atom_weights)*(-1),max(atom_weights)]
    canvas = mapvalues2mol(test_mol, atom_weights, bond_weights,color=colorscale,value_lims=limit)
    img = transform2png(canvas.GetDrawingText())
    display(img)

for nmol, data in enumerate(test_loader):

    # get the entry from the dataloader
    data = data.to(device)

    # get the most likely prediction of the model
    output = model(data.x, data.edge_index, data.edge_attr, data.batch)

    output_prob = F.softmax(output, dim=1)
    preds = output_prob.argmax(dim=1)

    if PLOT_UGRADCAM or len(input_molecules)==1:
        print(f'[INFO   ] Plotting UGrad-CAMs for molecule #{nmol+1}')

    for position, receptor  in enumerate(receptors):
        # Retrieve the prediction per pair
        pred = preds[position].item()
        prob = output_prob[position,pred].item()
        final_results_df.iloc[nmol,position] = output_prob[position,1].item()
       
        if NAME_OVERRIDE:
            name = f'molecule_{nmol+1}'
        else:
            name = molecules[nmol]
        
        if PLOT_UGRADCAM or len(input_molecules)==1:
            print(f'[INFO   ]   - TAS2R{receptor}')
            # Get the gradient of the output with respect to the parameters of the model 
            if position + 1 == len(receptors):
                output[position,pred].backward()
            else:
                output[position,pred].backward(retain_graph=True)
            
            # pull the gradients out of the model
            gradients = model.get_activations_gradient()
            # get the activations of the last convolutional layer
            activations = model.get_activations(data.x)
            if rec_to_inv == receptor:
                plot_on_mol(molecules[nmol],name,receptor,pred,activations,gradients)

Apply Applicability Domain and Ground Truth Check to the input molecule

In [ ]:


final_results_df = final_results_df.round(decimals=2)
final_results_df.index = molecules
# CHECK IF ALREADY KNOWN / INCOMPLETE -> 1/0 on know and prediction on unknown
# TO REMOVE THIS CHECK set ground_truth to FALSE
if GT:
    unk = ['Unknown'] * len(molecules)
    final_results_df.insert(loc=0, column='Ground Truth', value=unk)
    # Importing dataset for checks
    tdf = pd.read_csv('../data/dataset.csv', header=0, index_col=0)
    tdf.columns = tdf.columns.astype(int)

    # Check if input smiles are already present in ground truth dataset
    # Discriminate between fully known pairs, incomplete pairs and unknown
    std_known_smiles = tdf.loc[tdf.index.isin(molecules)]
    std_incomplete_smiles = std_known_smiles[std_known_smiles.isna().any(axis=1)]
    std_fullyknown_smiles = std_known_smiles.dropna(how='any')

    inc = ['Partially Known'] * len(std_incomplete_smiles.index)
    std_incomplete_smiles['Ground Truth'] = inc

    known = ['Fully Known'] * len(std_fullyknown_smiles)
    std_fullyknown_smiles['Ground Truth'] = known

    final_results_df.update(std_incomplete_smiles)
    final_results_df.update(std_fullyknown_smiles)

# CHECK if in Applicability Domain
AD_file = './src/AD.pkl'
check_AD = [TestAD(smi, filename=AD_file, verbose = False, sim_threshold=0.2, neighbors = 5, metric = "tanimoto") for smi in molecules]
test       = [i[0] for i in check_AD]
final_results_df.insert(loc=0, column='Check AD', value=test)

col = final_results_df.pop('Ground Truth')
final_results_df.insert(0, col.name, col)
final_results_df.insert(loc=0, column='Standardized SMILES',value=final_results_df.index)
final_results_df = final_results_df.reset_index(drop=True)

final_results_df.to_csv("GCN_output.csv", sep=",")
print('[DONE   ] Prediction and explanation tasks completed.')

## Results

Show the table with the <b> final results </b>

Every prediction displayed is the probability of the bind of each molecule to each receptor, from 0 (no-bind) to 1 (bind)

The Applicability Domain column shows if the input molecule is similar enough to the ones in the training dataset. If the check returns FALSE it is strongly advised to not consider the prediction for that molecule as reliable

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(final_results_df)